In [12]:
# Import libraries
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from urllib.request import urlopen, Request
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [18]:
# Parameters 
n = 3 #the # of article headlines displayed per ticker
tickers = ['PBR','MSFT']

# Get Data
finwiz_url = 'https://finviz.com/quote.ashx?t='
news_tables = {}

In [19]:
for ticker in tickers:
    url = finwiz_url + ticker
    req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 
    resp = urlopen(req)    
    html = BeautifulSoup(resp, features="lxml")
    news_table = html.find(id='news-table')
    news_tables[ticker] = news_table

try:
    for ticker in tickers:
        df = news_tables[ticker]
        df_tr = df.findAll('tr')
    
        print ('\n')
        print ('Recent News Headlines for {}: '.format(ticker))
        
        for i, table_row in enumerate(df_tr):
            a_text = table_row.a.text
            td_text = table_row.td.text
            td_text = td_text.strip()
            print(a_text,'(',td_text,')')
            if i == n-1:
                break
except KeyError:
    pass

# Iterate through the news
parsed_news = []
for file_name, news_table in news_tables.items():
    for x in news_table.findAll('tr'):
        text = x.a.get_text() 
        date_scrape = x.td.text.split()

        if len(date_scrape) == 1:
            time = date_scrape[0]
            
        else:
            date = date_scrape[0]
            time = date_scrape[1]

        ticker = file_name.split('_')[0]
        
        parsed_news.append([ticker, date, time, text])



Recent News Headlines for PBR: 
Petroleo Brasileiro S.A.- Petrobras (PBR) Q3 2020 Earnings Call Transcript ( Oct-30-20 03:30AM )
Petrobras (PBR) Postpones Parque das Baleias Project by a Year ( Oct-26-20 12:03PM )
Petrobras Announces Final Results Of Its Cash Tender Offers ( Oct-22-20 04:01PM )


Recent News Headlines for MSFT: 
4 Reasons Sony's Stock Could Still Head Higher ( Nov-01-20 10:00AM )
Better Buy: Activision Blizzard vs. Microsoft ( 07:00AM )
3 Top Renewable Energy Stocks to Buy No Matter Who Wins the Election ( 07:00AM )


In [20]:
analyzer = SentimentIntensityAnalyzer()

In [21]:
columns = ['Ticker', 'Date', 'Time', 'Headline']
news = pd.DataFrame(parsed_news, columns=columns)
scores = news['Headline'].apply(analyzer.polarity_scores).tolist()

df_scores = pd.DataFrame(scores)
news = news.join(df_scores, rsuffix='_right')


# View Data 
news['Date'] = pd.to_datetime(news.Date).dt.date

unique_ticker = news['Ticker'].unique().tolist()
news_dict = {name: news.loc[news['Ticker'] == name] for name in unique_ticker}

values = []
for ticker in tickers: 
    dataframe = news_dict[ticker]
    dataframe = dataframe.set_index('Ticker')
    dataframe = dataframe.drop(columns = ['Headline'])
    print ('\n')
    print (dataframe.head())
    
    mean = round(dataframe['compound'].mean(), 2)
    values.append(mean)
    
df = pd.DataFrame(list(zip(tickers, values)), columns =['Ticker', 'Mean Sentiment']) 
df = df.set_index('Ticker')
df = df.sort_values('Mean Sentiment', ascending=False)
print ('\n')
print (df)



              Date     Time    neg    neu    pos  compound
Ticker                                                    
PBR     2020-10-30  03:30AM  0.000  1.000  0.000    0.0000
PBR     2020-10-26  12:03PM  0.208  0.792  0.000   -0.2732
PBR     2020-10-22  04:01PM  0.000  1.000  0.000    0.0000
PBR     2020-10-20  10:24PM  0.000  0.796  0.204    0.3182
PBR     2020-10-20  10:24PM  0.000  0.796  0.204    0.3182


              Date     Time    neg    neu    pos  compound
Ticker                                                    
MSFT    2020-11-01  10:00AM  0.000  1.000  0.000    0.0000
MSFT    2020-11-01  07:00AM  0.000  0.633  0.367    0.4404
MSFT    2020-11-01  07:00AM  0.123  0.391  0.486    0.6705
MSFT    2020-10-31  12:27PM  0.240  0.760  0.000   -0.6249
MSFT    2020-10-31  10:39AM  0.145  0.645  0.210    0.2023


        Mean Sentiment
Ticker                
MSFT              0.12
PBR               0.02
